In [2]:
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import Scatter, Layout
from scipy.stats import binom
from ipywidgets import interact, IntSlider

init_notebook_mode(connected=True)

@interact(dice=IntSlider(min=1, max=12), tn=IntSlider(min=2, max=6))
def plot_hits(dice, tn):
    probabilities = [round(binom.cdf(dice-h, dice, (tn-1)/6), 3) * 100 for h in range(dice+1)]
    x_range = list(range(0, dice+1))
    layout = Layout(
                 title='Probability of N its for a given TN and dice pool',
                 xaxis={'title': 'Number of hits',
                        'ticktext': ['{}+'.format(n) for n in x_range],
                        'tickvals': x_range
                       },
                 yaxis={'title': 'Probability (%)'}
             )
    iplot({'data': [Scatter(x=x_range, y=probabilities)], 'layout': layout})